In [1]:
import sys
sys.path.append('../src')
from models import *
from strategies import *
from custom_datasets import *
import numpy as np
np.random.seed(0)
import tqdm

import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
import torch
torch.cuda.empty_cache()
import torch.nn as nn

from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
import pandas as pd

import time
import json
import wandb
from unet_model import *


In [2]:
main_path = "/root/Master_Thesis/"
dataframes_path = main_path + "data/dataframes/"
sam_path = main_path + "sam/sam_vit_h_4b8939.pth"
voters_path = main_path + "scripts/notebooks/trained_models/voters/"
notebooks_path = main_path + "scripts/notebooks/"
expirements_path = main_path+"expirements/"

In [3]:
# df_name = "brain_df"
# train_df = pd.read_csv(dataframes_path+"brain_df_train.csv")
# test_df = pd.read_csv(dataframes_path+"brain_df_test.csv")

df_name = "lung_tumor_df"
train_df = pd.read_csv(dataframes_path+"lung_df_train.csv")
test_df = pd.read_csv(dataframes_path+"lung_df_test.csv")

# df_name = "flood_df"
# train_df = pd.read_csv(dataframes_path+"flood_df_train.csv")
# test_df = pd.read_csv(dataframes_path+"flood_df_test.csv")

In [4]:
len(test_df)

495

In [5]:
# with open('params.json') as f:
#     params = json.load(f)
#     print(params)

In [6]:
len(train_df)

1153

In [7]:
params = {'n_epoch': 35,
          'train_args':{'batch_size': 16, 'num_workers': 1},
          'test_args':{'batch_size': 512, 'num_workers': 1},
          'optimizer_args':{'lr': 5e-3, 'momentum': 0.9},
          'use_sam': False,
          'use_predictor': False,
          'use_generator': False,
          'init_set_size': 200,
          'rounds': 23,
          "activate_sam_at_round":1, 
          "img_size":(128, 128),
          "voting" : False,
          "pre_trained":True}

if params["init_set_size"] == len(train_df):
    params["training_type"] = "no_active"
elif not params["use_sam"]:
    params["training_type"] = "no_sam"
elif params["voting"]:
    params["training_type"] = "voters"
else:
    params["training_type"]="withSAM_NoVoting"

if params["training_type"] == "no_active":
    if params["pre_trained"]:
        params["model_path"] = f'{notebooks_path}trained_models/no_active/'
    else:
        params["model_path"] = f'{notebooks_path}trained_models/no_active/'
else:
    params["model_path"] = f'{notebooks_path}trained_models/{params["training_type"]}/{params["init_set_size"]}'
    
params['test_set_size'] = len(test_df)
params['df'] = df_name
params['query_num'] = int(0.05 * params['init_set_size'])
if params['query_num'] == 0:
    params['query_num'] = 1
params["strategy"] = "MarginSampling"

if params["training_type"] == "voters":
    params["model_path"] = f'{params["model_path"]}/voters_{params["img_size"][0]}'

In [8]:
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="baseline",
    
#     notes = "SAM100-Voting",

#     # track hyperparameters and run metadata
#     config=params    
# )

In [9]:
if params['use_sam']:
    sam = SAMOracle(checkpoint_path=sam_path, img_size=params["img_size"])
else:
    sam =None

In [10]:
if params["pre_trained"]:
    model = smp.create_model('Unet', encoder_name='resnet34', in_channels=3, classes = 1)
else:
    model = UNet(n_channels=3, n_classes=1, bilinear=True)

In [11]:
init_path = ""

if params["training_type"] == "voters":
    init_path = params["model_path"] + '_0/main_Unet.pt'

elif params["training_type"] == "withSAM_NoVoting":
    init_path = params["model_path"] + '/main_Unet_128_0.pt'
    
elif params["training_type"] == "no_sam":
    init_path = f'{params["model_path"]}/Active_{params["init_set_size"]}_{0}_no_sam_{params["img_size"][0]}_Unet.pt'
    
if len(init_path) > 0:
    if not os.path.isfile(init_path):
        init_dir = os.path.dirname(init_path)
        if not os.path.exists(init_dir):
            os.makedirs(init_dir)
        torch.save(model.state_dict(), init_path)
    init_state_Unet = torch.load(init_path)
    model.load_state_dict(init_state_Unet)
    


net = Net(model, params, device = torch.device("cuda"))

In [12]:
init_path

'/root/Master_Thesis/scripts/notebooks/trained_models/no_sam/200/Active_200_0_no_sam_128_Unet.pt'

In [13]:
def get_data(handler, train_df, test_df):
    return Data(train_df["images"].to_list(), train_df["masks"].to_list(), test_df["images"].to_list(), test_df["masks"].to_list(), handler, img_size=params["img_size"], df=train_df, path= main_path+"/data/processed/", use_sam=params['use_sam'])

In [14]:
data = get_data(Handler, train_df, test_df)
data.initialize_labels(params["init_set_size"])

In [15]:
strategy = MarginSampling(dataset=data, net=net, sam=sam, params=params)
params["strategy"] = "MarginSampling"

In [16]:
params["model_path"]

'/root/Master_Thesis/scripts/notebooks/trained_models/no_sam/200'

In [17]:
print("Round 0")
rd = 1
logs =[]
main_path = ""
if params["training_type"] == "no_sam":
    main_path = f'{params["model_path"]}/Active_{params["init_set_size"]}_{rd}_no_sam_{params["img_size"][0]}_Unet.pt'

elif params["training_type"] == "voters":
    main_path = f'{params["model_path"]}_{rd}/main_Unet.pt'

elif params["training_type"] == "withSAM_NoVoting":
    main_path = f'{params["model_path"]}/main_Unet_{params["img_size"][0]}_{rd}.pt'

# if len(main_path)>0:
#     if not os.path.isfile(main_path):
#         strategy.train()
#         main_dir = os.path.dirname(main_path)
#         if not os.path.exists(main_dir):
#             os.makedirs(main_dir)
#         torch.save(strategy.net.net.state_dict(), main_path)
#         print("Saved : " + main_path)
#     else:
#         strategy.net.net.load_state_dict(torch.load(main_path))
print("Uncomment the block above")        
strategy.train()
    
logits, mask_gt = strategy.predict(data.get_test_data())
iou_score, accuracy, precision, recall, f1_score = data.cal_test_metrics(logits, mask_gt )
# wandb.log({"iou_score" : iou_score, "accuracy" : accuracy, "precision" : precision, "recall" : recall, "f1_score" : f1_score})
logs.append(f"Round 0 testing metrics: iou_score = {iou_score:.2f}, accuracy = {accuracy:.2f}, precision = {precision:.2f}, recall = {recall:.2f}, f1_score = {f1_score:.2f}")
print(logs[0])

for rd in range(1, params["rounds"]):
    print(f"Round {rd}")

    # query
    print("Querying")
    query_idxs = strategy.query(params["query_num"])
    print(query_idxs)
    # update labels
    if params["use_sam"] and rd >= params["activate_sam_at_round"]:
        print("Updating with sam")
        if params["training_type"] == "voters":
            masks = strategy.update_voting(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"], round=rd)
        
        elif params["training_type"] == "withSAM_NoVoting":
            masks = strategy.update(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"], round=rd)
        # else:
            # masks = strategy.update_weighted_voting(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"], round=rd)
    else:
        print("Updating without sam")
        strategy.update(query_idxs)
    
    print("Reset and train")
    if params["training_type"] == "no_sam":
        main_path = f'{params["model_path"]}/Active_{params["init_set_size"]}_{rd+1}_no_sam_{params["img_size"][0]}_Unet.pt'

    elif params["training_type"] == "voters":
        main_path = f'{params["model_path"]}_{rd+1}/main_Unet.pt'

    elif params["training_type"] == "withSAM_NoVoting":
        main_path = f'{params["model_path"]}/main_Unet_{params["img_size"][0]}_{rd+1}.pt'

    # if not os.path.isfile(main_path):
    #     # strategy.net.net.load_state_dict(init_state_Unet)
    #     strategy.train()
    #     main_dir = os.path.dirname(main_path)
    #     if not os.path.exists(main_dir):
    #         os.makedirs(main_dir)
    #     torch.save(strategy.net.net.state_dict(), main_path)
    #     print("Saved : " + main_path)
    # else:
    #     # strategy.net.net.load_state_dict(torch.load(main_path))
    print("uncomment the block above")
    strategy.net.net.load_state_dict(init_state_Unet)
    strategy.train()    

    # calculate accuracy
    logits, maks_gt = strategy.predict(data.get_test_data())
    iou_score, accuracy, precision, recall, f1_score = data.cal_test_metrics(logits, mask_gt )
    # wandb.log({"iou_score" : iou_score, "accuracy" : accuracy, "precision" : precision, "recall" : recall, "f1_score" : f1_score})
    logs.append(f"Round {rd} testing metrics: iou_score = {iou_score:.2f}, accuracy = {accuracy:.2f}, precision = {precision:.2f}, recall = {recall:.2f}, f1_score = {f1_score:.2f}")
    print(logs[rd])
    
params['logs'] = logs
# wandb.finish()

Round 0
Uncomment the block above


100%|███████████████████████████████████████████████████| 35/35 [01:36<00:00,  2.77s/it, loss=0.235]


Round 0 testing metrics: iou_score = 0.63, accuracy = 1.00, precision = 0.87, recall = 0.69, f1_score = 0.77
Round 1
Querying


KeyboardInterrupt: 

In [ ]:
# from pathlib import Path
# Path("./my/directory").mkdir(parents=True, exist_ok=True)

import os



In [ ]:
os.getcwd()

'/root/Master_Thesis/scripts/notebooks'

In [ ]:
# for dirname, _, filenames in os.walk(expirements_path):
#     filename = "expirement_{}.json".format(len(filenames))
#     file_path = os.path.join(dirname, filename)
#     with open(file_path, 'w') as f:
#         json.dump(params, f)
#         print(filename)

# UNet

In [ ]:
# strategy.net = net_1

In [ ]:
# model_1 = torch.load('model_1.pt')
# strategy.net.net.load_state_dict(model_1)
# strategy.net.clf = strategy.net.net.to(torch.device("cuda:1"))
# mask_1 = strategy.predict(strategy.dataset.handler([strategy.dataset.df["images"][idx]], [strategy.dataset.df["masks"][idx]]))[0]
# mask_1 = (mask_1.squeeze().cpu().sigmoid()> 0.5).float()

In [ ]:
# model_2 = torch.load('model_2.pt')
# strategy.net.net.load_state_dict(model_2)
# strategy.net.clf = strategy.net.net.to(torch.device("cuda:1"))
# mask_2 = strategy.predict(strategy.dataset.handler([strategy.dataset.df["images"][idx]], [strategy.dataset.df["masks"][idx]]))[0]
# mask_2 = (mask_2.squeeze().cpu().sigmoid()> 0.5).float()

In [ ]:
# model_3 = torch.load('model_3.pt')
# strategy.net.net.load_state_dict(model_3)
# strategy.net.clf = strategy.net.net.to(torch.device("cuda:1"))
# mask_3 = strategy.predict(strategy.dataset.handler([strategy.dataset.df["images"][idx]], [strategy.dataset.df["masks"][idx]]))[0]
# mask_3 = (mask_3.squeeze().cpu().sigmoid()> 0.5).float()

# FPN

In [ ]:
# strategy.net = net_2

In [ ]:
# model_4 = torch.load('model_4.pt')
# strategy.net.net.load_state_dict(model_4)
# strategy.net.clf = strategy.net.net.to(torch.device("cuda:1"))
# mask_4 = strategy.predict(strategy.dataset.handler([strategy.dataset.df["images"][idx]], [strategy.dataset.df["masks"][idx]]))[0]
# mask_4 = (mask_4.squeeze().cpu().sigmoid()> 0.5).float()

In [ ]:
# def predict(model_states:list, idx):
#     masks = []
#     for state in model_states:
#         model = torch.load(state)
#         strategy.net.net.load_state_dict(model)
#         strategy.net.clf = strategy.net.net.to(torch.device("cuda:1"))
#         mask = strategy.predict(strategy.dataset.handler([strategy.dataset.df["images"][idx]], [strategy.dataset.df["masks"][idx]]))[0]
#         mask = (mask.squeeze().cpu().sigmoid()> 0.5).float()
#         masks.append(mask)
#     return masks

In [ ]:
# masks = predict(["model_1.pt", "model_2.pt", "model_3.pt", "model_4.pt"], 2197)

In [ ]:
# import supervision as sv
# imgs = [mask_1, mask_2, mask_3, mask_4]
# sv.plot_images_grid(
#     images=imgs,
#     grid_size=(1, len(imgs)),
#     # titles=['mask_1', 'mask_2', "mask_3","mask_4"]
# )